In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
import folium
import pickle

In [2]:
taxi_data = pd.read_csv('trip_data.csv')

In [3]:
taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [4]:
taxi_data[['tpep_pickup_datetime', 'tpep_dropoff_datetime']]

,tpep_pickup_datetime,tpep_dropoff_datetime
0,2015-01-15 19:05:39,2015-01-15 19:23:42
1,2015-01-10 20:33:38,2015-01-10 20:53:28
2,2015-01-10 20:33:38,2015-01-10 20:43:41
3,2015-01-10 20:33:39,2015-01-10 20:35:31
4,2015-01-10 20:33:39,2015-01-10 20:52:58
...,...,...
12748981,2015-01-10 19:01:44,2015-01-10 19:05:40
12748982,2015-01-10 19:01:44,2015-01-10 19:07:26
12748983,2015-01-10 19:01:44,2015-01-10 19:15:01
12748984,2015-01-10 19:01:44,2015-01-10 19:17:03


In [5]:
# Make time names more clear
taxi_data = taxi_data.rename(columns={
    'tpep_pickup_datetime': 'pickup_time',
    'tpep_dropoff_datetime': 'drop_off_time'
})

In [6]:
taxi_data

,VendorID,pickup_time,drop_off_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [7]:
# Convert columns to datetime format
taxi_data['pickup_time'] = pd.to_datetime(taxi_data['pickup_time'])
taxi_data['drop_off_time'] = pd.to_datetime(taxi_data['drop_off_time'])

# Display the data types to confirm the changes
taxi_data.dtypes

VendorID                          int64
pickup_time              datetime64[ns]
drop_off_time            datetime64[ns]
passenger_count                   int64
trip_distance                   float64
pickup_longitude                float64
pickup_latitude                 float64
RateCodeID                        int64
store_and_fwd_flag               object
dropoff_longitude               float64
dropoff_latitude                float64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

In [8]:
taxi_data['pickup_time']

0          2015-01-15 19:05:39
1          2015-01-10 20:33:38
2          2015-01-10 20:33:38
3          2015-01-10 20:33:39
4          2015-01-10 20:33:39
                   ...        
12748981   2015-01-10 19:01:44
12748982   2015-01-10 19:01:44
12748983   2015-01-10 19:01:44
12748984   2015-01-10 19:01:44
12748985   2015-01-10 19:01:45
Name: pickup_time, Length: 12748986, dtype: datetime64[ns]

In [9]:
taxi_data.isna().sum()

VendorID                 0
pickup_time              0
drop_off_time            0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
RateCodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    3
total_amount             0
dtype: int64

In [10]:
# Drop three duplcates
taxi_data.dropna()

,VendorID,pickup_time,drop_off_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [11]:
taxi_data.duplicated().sum()

383

In [12]:
duplicate_rows = taxi_data[taxi_data.duplicated()]

### These anamolies should be removed as they have no data worth keeping

In [13]:
duplicate_rows

,VendorID,pickup_time,drop_off_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
248333,2,2015-01-05 09:39:49,2015-01-05 09:39:52,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
248334,2,2015-01-05 09:39:49,2015-01-05 09:39:52,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
300143,2,2015-01-05 09:39:49,2015-01-05 09:39:52,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
300144,2,2015-01-05 09:39:49,2015-01-05 09:39:52,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
300145,2,2015-01-05 09:39:49,2015-01-05 09:39:52,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12242582,2,2015-01-14 13:20:13,2015-01-14 13:20:33,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.0,0.0,0.5,0.0,0.0,0.3,2.8
12242583,2,2015-01-14 13:20:13,2015-01-14 13:20:33,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.0,0.0,0.5,0.0,0.0,0.3,2.8
12242584,2,2015-01-14 13:20:13,2015-01-14 13:20:33,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.0,0.0,0.5,0.0,0.0,0.3,2.8
12242585,2,2015-01-14 13:20:13,2015-01-14 13:20:33,1,0.0,0.0,0.0,1,N,0.0,0.0,2,2.0,0.0,0.5,0.0,0.0,0.3,2.8


In [14]:
taxi_data.drop_duplicates()

,VendorID,pickup_time,drop_off_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [15]:
# This column is irelevant
taxi_data.drop(columns=['store_and_fwd_flag'], inplace=True)

In [16]:
# Create trip duration column
taxi_data['Trip_Duration'] =  taxi_data['drop_off_time'] - taxi_data['pickup_time']

In [17]:
taxi_data['Trip_Duration'] = pd.to_timedelta(taxi_data['Trip_Duration'])
# Calculate total seconds
taxi_data['Total_Seconds'] = taxi_data['Trip_Duration'].dt.total_seconds()

# Convert seconds to minutes
taxi_data['Trip_Duration_in Minutes'] = taxi_data['Total_Seconds'] / 60

# Drop the 'Total Seconds' column if it's not needed
taxi_data = taxi_data.drop(columns=['Total_Seconds'])


In [18]:
# Duplicate Column removed
taxi_data.drop(columns=['Trip_Duration'], inplace=True)

In [19]:
# Mapping of numbers to payment method names
payment_method_mapping = {
    1: 'Credit card',
    2: 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided trip'
}

# Replace the numbers with the corresponding names
taxi_data['PaymentMethod'] = taxi_data['payment_type'].replace(payment_method_mapping)

In [20]:
taxi_data.head()

,VendorID,pickup_time,drop_off_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Trip_Duration_in Minutes,PaymentMethod
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,18.050000,Credit card
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80,19.833333,Credit card
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,10.050000,Cash
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,1.866667,Cash
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30,19.316667,Cash


In [21]:
taxi_data.to_pickle('wrangling_taxi.pkl')

In [22]:
taxi_data.to_csv('cleaned_taxi_data')